<a href="https://colab.research.google.com/github/nickdlc/CSc448-Projects/blob/main/Assignment3/Equation-of-Slime.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

# Loading the Dataset

In [3]:
# Display the first 15 rows of the dataset
df = pd.read_csv('https://raw.githubusercontent.com/profmcnich/example_notebook/main/science_data_large.csv')
df.head(15)

,Temperature °C,Mols KCL,Size nm^3
0,469,647,6.244743e+05
1,403,694,5.779610e+05
2,302,975,6.196847e+05
3,779,916,1.460449e+06
4,901,18,4.325726e+04
5,545,637,7.124634e+05
6,660,519,7.006960e+05
7,143,869,2.718260e+05
8,89,461,8.919803e+04
9,294,776,4.770210e+05


In [4]:
# Display a summary of the dataset
df.describe()

,Temperature °C,Mols KCL,Size nm^3
count,1000.000000,1000.000000,1.000000e+03
mean,500.500000,471.530000,5.086111e+05
std,288.819436,288.482872,4.474838e+05
min,1.000000,1.000000,1.611429e+01
25%,250.750000,226.750000,1.298267e+05
50%,500.500000,459.500000,3.827182e+05
75%,750.250000,710.250000,7.603211e+05
max,1000.000000,1000.000000,1.972127e+06


# Splitting the Dataset

In [10]:
from sklearn.model_selection import train_test_split

# Determine the features and label for the dataset
X = df.drop('Size nm^3', axis=1)
y = df['Size nm^3']

# Split the data for training (90%) and testing (10%)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

# Linear Regression Model

In [11]:
from sklearn.linear_model import LinearRegression

# Train a linear regression model on the training data
linreg = LinearRegression()
linreg.fit(X_train, y_train)

LinearRegression()

In [12]:
from sklearn.metrics import confusion_matrix

# Use the test data to predict values
y_pred = linreg.predict(X_test)

linreg.score(X_test, y_test)

0.8552472077276095

# Cross Validation

# Polynomial Regression